In [1]:
# Install Pytorch & other libraries
%pip install "torch>=2.4.0" tensorboard torchvision

# Install Gemma release branch from Hugging Face
%pip install "transformers>=4.51.3"

# Install Hugging Face libraries
%pip install  --upgrade \
  "datasets==3.3.2" \
  "accelerate==1.4.0" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.45.3" \
  "trl==0.15.2" \
  "peft==0.14.0" \
  "pillow==11.1.0" \
  protobuf \
  sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      

In [2]:
#Mount the notebook on to the google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Set the working directory to dl_project_fall_2025
import os
os.chdir("/content/drive/MyDrive/DL_Project_2025/dl_project_fall_2025")

# Auto relaod doesnt work in google colab, so you can use reload to reload your function calls
from importlib import reload


Mounted at /content/drive


In [5]:
!git add .

In [6]:
!git status

On branch google_colab_development
Your branch is up to date with 'origin/google_colab_development'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   __init__.py
	deleted:    notebooks/imageclef_captioning.ipynb
	new file:   src/llmft/__pycache__/__init__.cpython-312.pyc
	new file:   src/llmft/training/__pycache__/__init__.cpython-312.pyc
	new file:   src/llmft/training/__pycache__/trainer.cpython-312.pyc
	modified:   src/llmft/training/trainer.py



In [9]:
!git config --global user.email "7yashwanth7@gmail.com"
!git config --global user.name "Yashwanth Loganathan"

In [10]:
!git commit -m "first commit"

[google_colab_development 794454a] first commit
 6 files changed, 6 insertions(+), 277 deletions(-)
 create mode 100644 __init__.py
 delete mode 100644 notebooks/imageclef_captioning.ipynb
 create mode 100644 src/llmft/__pycache__/__init__.cpython-312.pyc
 create mode 100644 src/llmft/training/__pycache__/__init__.cpython-312.pyc
 create mode 100644 src/llmft/training/__pycache__/trainer.cpython-312.pyc


In [13]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address


In [14]:
!git pull

Already up to date.


In [15]:
!git push

fatal: could not read Username for 'https://github.com': No such device or address


In [44]:
from google.colab import userdata

# Retrieve the GitHub Token from Colab secrets
GH_TOKEN = userdata.get('git_token') # Ensure you stored your PAT under the secret name 'GH_TOKEN'

# Configure Git to use the PAT directly in the remote URL for the 'origin'
!git remote set-url origin https://{GH_TOKEN}@github.com/7yashwanth7/dl_project_fall_2025.git

print("Git remote 'origin' URL updated successfully with GitHub Personal Access Token.")

Git remote 'origin' URL updated successfully with GitHub Personal Access Token.


In [48]:
!git status

On branch google_colab_development
Your branch is ahead of 'origin/google_colab_development' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   src/llmft/training/callbacks.py
	modified:   src/llmft/training/trainer.py

no changes added to commit (use "git add" and/or "git commit -a")


In [49]:
!git add .

In [50]:
!git status

On branch google_colab_development
Your branch is ahead of 'origin/google_colab_development' by 1 commit.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   src/llmft/training/callbacks.py
	modified:   src/llmft/training/trainer.py

